## Environment Testing with Deep Reinforcement Learning Methods

In [3]:
from pprint import pprint

In [1]:
from envs.dining_hall_env import DiningHallEnv
# SETUP PARAMETERS OF DINING HALL
num_diners = 100
num_dining_halls = 5
num_dining_days = 7 # important for establishing specials probability for halls
num_dining_times = 10
env = DiningHallEnv(num_diners=num_diners, num_dining_halls=num_dining_halls, num_dining_days=num_dining_days, num_dining_times=num_dining_times)

DINING TIMES:


In [4]:
# Inspect Observation and Action Space
pprint(env.action_space)
pprint(env.observation_space)
# We can also inspect (although our agent won't use it) the specials probability (useful for constructing some optimal policies)
pprint(env.special_probs)

MultiDiscrete([5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5])
Dict(current_day:Discrete(7), unavailable_halls:MultiDiscrete([6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6]))
array([[0.20497554, 0.60269791, 0.5279855 , 0.14783359, 0.07459853,
        0.29877446, 0.87111719],
       [0.45654221, 0.8731332 , 0.48483085, 0.42216488, 0.80322738,
        0.20052952, 0.38965143],
       [0.89529731, 0.87672316, 0.82578997, 0.4252813 , 0.45541589,
        0.6836752 , 0.19428943],
       [0.51915442, 0.3650452 , 0.62922959, 0.03607705, 0.57794386,
        0.63314045, 0.96676962],
       [0.72136064, 0.96062419, 0.3544264 , 0.93021577, 0.20732983,
        0.7226198 , 0.36804707]])


In [14]:
from gym.envs.registration import register
import gym
register(
    id="DiningHall-v0",
    entry_point=DiningHallEnv
)


In [13]:
# testing if environment got registered
test_env = gym.make('ai_diner/DiningHall-v0',num_diners=num_diners, num_dining_halls=num_dining_halls, num_dining_days=num_dining_days, num_dining_times=num_dining_times)
test_env.action_space.sample()

DINING TIMES:


array([1, 0, 4, 1, 2, 2, 1, 4, 0, 3, 0, 1, 0, 0, 2, 2, 0, 2, 4, 2, 4, 0,
       4, 2, 4, 2, 0, 1, 2, 2, 4, 0, 2, 0, 0, 1, 1, 0, 2, 3, 0, 4, 1, 1,
       3, 4, 3, 3, 0, 1, 1, 4, 4, 2, 2, 3, 4, 4, 4, 2, 1, 4, 2, 3, 0, 1,
       0, 1, 3, 4, 2, 0, 4, 4, 0, 1, 0, 0, 4, 1, 1, 0, 2, 0, 2, 3, 1, 3,
       4, 0, 3, 0, 3, 1, 4, 4, 3, 3, 1, 2])

### We are using off-the-shelf deep RL models provided by the Stable-Baselines library. Due to the environment setup, we are limited to only two deep RL Methods: A2C and PPO.

In [5]:
# Make sure our environment is compatible with Stable-Baselines
from stable_baselines3.common.env_checker import check_env
check_env(env, warn=True)


In [6]:
from stable_baselines3 import A2C, PPO
a2c_model = A2C("MultiInputPolicy", env, verbose=1)
ppo_model = PPO("MultiInputPolicy", env, verbose=1)
# model.learn(total_timesteps=10000, log_interval=4)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [52]:

a2c_model.learn()



model.save("a2c_diner")

del model # remove to demonstrate saving and loading

model = A2C.load("a2c_diner")

In [53]:
obs = env.reset()
i = 0
while True:
    action, _states = model.predict(obs)
    day = obs["current_day"]
    print(day, action, rewards)
    obs, rewards, dones, info = env.step(action)
    i += 1
    if i > 10:
        break

0 [0 2 2 4 1 2 3 1 2 2 4 0 2 4 2 1 1 3 3 4 1 2 0 3 2 1 0 0 1 2 1 1 0 0 1 0 4
 1 3 4 1 0 2 3 4 2 2 0 0 0 1 3 3 3 1 3 0 2 1 2 4 2 4 1 2 1 2 4 3 1 2 0 3 1
 4 4 1 4 4 1 4 4 0 4 0 3 4 2 1 3 1 2 2 0 0 0 4 2 4 0] 20.0
1 [1 0 4 3 3 2 1 4 1 4 4 4 2 4 0 1 4 2 2 0 2 0 0 3 2 0 0 3 1 2 2 4 4 0 4 2 1
 0 4 4 0 3 1 1 2 3 0 2 0 2 4 0 0 1 1 2 1 2 2 3 0 3 4 1 2 4 4 1 0 1 0 3 4 0
 0 4 2 3 3 0 1 3 4 0 0 4 0 4 0 4 3 1 2 0 0 0 1 1 2 2] -25773.427604301447
2 [1 1 4 2 3 0 0 2 1 3 3 4 4 2 4 2 3 1 3 1 2 4 4 2 2 4 4 0 2 3 3 3 3 1 4 4 4
 0 0 1 3 2 3 3 1 0 1 3 0 2 4 4 2 1 3 4 1 4 3 1 0 3 1 1 2 1 0 1 2 1 0 4 4 0
 4 0 3 3 0 4 4 0 2 1 4 4 0 0 1 1 3 1 0 3 0 2 2 3 1 4] -25574.097187210267
3 [1 2 0 4 2 0 1 1 3 0 4 2 3 4 0 1 2 4 3 0 1 2 4 1 0 2 3 2 3 2 4 3 2 0 1 1 1
 4 3 4 0 4 2 1 4 2 0 1 1 3 1 2 2 1 4 1 3 1 3 0 0 1 1 3 2 4 3 4 4 3 2 0 2 0
 4 4 1 2 0 4 0 4 4 4 3 2 4 1 0 0 3 1 2 4 4 0 2 1 2 0] -28593.08306872006
4 [2 2 0 0 1 2 1 1 3 4 4 3 2 4 0 3 3 2 2 3 2 0 2 0 0 4 3 0 1 4 3 4 2 2 1 1 4
 2 0 4 2 4 0 3 4 2 2 1 0 2 4 3 4 1 

In [54]:
from stable_baselines3 import PPO
model = PPO("MultiInputPolicy", env, verbose=1)
model.learn(total_timesteps=50000)
model.save("ppo_diner")

del model # remove to demonstrate saving and loading

model = PPO.load("ppo_diner")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 55   |
|    iterations      | 1    |
|    time_elapsed    | 36   |
|    total_timesteps | 2048 |
-----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 45            |
|    iterations           | 2             |
|    time_elapsed         | 89            |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 1.5525729e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -161          |
|    explained_variance   | 5.96e-08      |
|    learning_rate        | 0.0003        |
|    loss                 | 5.17e+10      |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.0007 

In [47]:
import numpy as np
obs = env.reset()
i = 0
print(np.argmax(info["special_probs"], axis=0))
while True:
    action, _states = model.predict(obs)
    day = obs["current_day"]
    print(day, action, rewards)
    obs, rewards, dones, info = env.step(action)
    i += 1
    if i > 10:
        break

[1 1 4 4 1 3 4]
0 [1] 10.0
1 [1] 20.0
2 [3] -990.0
3 [1] 20.0
4 [2] 20.0
5 [3] 10.0
6 [1] 20.0
0 [0] 20.0
1 [1] 20.0
2 [2] 20.0
3 [4] 10.0


In [41]:
import numpy as np
print(info["special_probs"])

[[0.68048739 0.58412807 0.94703602 0.42349863 0.69825954 0.0272923
  0.38269711]
 [0.9148012  0.88536284 0.80473278 0.33527957 0.86038846 0.65524026
  0.55762551]
 [0.07373496 0.35836562 0.06491196 0.87533902 0.04280173 0.40592098
  0.41552788]
 [0.49079895 0.10607619 0.6842602  0.35562208 0.37553194 0.66906777
  0.70639979]
 [0.2722031  0.17432987 0.96813667 0.881014   0.68859568 0.46819329
  0.94272305]]
[1 1 4 4 1 3 4]
